In [144]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width=300)

numcores=8

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
library(jtools)

`%!in%` = Negate(`%in%`)

setDTthreads(numcores)

# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [145]:
#diagnosis codes
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=52850&ver=26 and https://www.aapc.com/codes/cpt-codes-range/93451-93533/10
angio_codes=c(93451,93452,93453,93454,93455,93456,93457,93458,93459,93460,93461,93462,93463,93464
              ,93503,93505,93530,93531,93532,93533)
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleid=57326&ver=13&keyword=electrocardiogram&keywordType=starts&areaId=all&docType=NCA,CAL,NCD,MEDCAC,TA,MCD,6,3,5,1,F,P&contractOption=all&sortBy=relevance&bc=1
ecg_codes=c(93000,93005,93010,93040,93041,93042)
#from https://scct.org/page/CardiacCTCodes include CTangio
cardiac_ct_codes=c(75571,75572,75573,75574)
#from https://cardiacmri.com/tech-guide/cpt-codes-relevant-to-cardiac-mri/
cardiac_mri_codes=c(75557,75559,75561,75563,75565)
# from https://medicarepaymentandreimbursement.com/2011/07/cardiovascular-stress-testing-cpt-93015.html and https://www.aapc.com/codes/cpt-codes-range/93000-93050/
stress_test_codes=c(93015,93016,93017,93018)
#from https://www.aapc.com/codes/cpt-codes-range/93303-93356/20     includes stress echo
echocardiography_codes=c(93303,93304,93306,93307,93308,93312,93313,93314,93315,93316,93317,93318
                         ,93320,93321,93325,93350,93351,93356,93352,93355,93356)
#from https://www.aapc.com/codes/cpt-codes-range/92920-92979/ and https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=57479#:~:text=CPT%20codes%2092928%2C%2092933%2C%2092929,are%20assigned%20to%20APC%200104.    includes balloon and stent
angioplasty_codes=c(92920,92921,92924,92925,92928,92929,92933,92934,92937,92938,92941,92943,92944
                    ,92973,92974,92975,92978,92979,93571,93572,"C9600","C9601","C9602","C9603"
                    ,"C9604","C9605","C9606","C9607","C9608")
#from https://www.medaxiom.com/clientuploads/webcast_handouts/Coding_for_CABG-Open_Heart_Procedures.pdf and https://www.aapc.com/codes/cpt-codes-range/33016-33999/10    did not include 33517-33530 since these are used in conjunction with 33533-33548 and not alone, did not include 33542,33545,33548 since these are also in conjunction )aneurismectomy and vsd resection
CABG_codes=c(33510,33511,33512,33513,33514,33516,33533,33534,33535,33536)
#from http://www.icd9data.com/2015/Volume1/390-459/430-438/default.htm and https://www.icd10data.com/ICD10CM/Codes/I00-I99/I60-I69/I63-
stroke_icd_9_codes=c(43301,43311,43321,43331,43381,43391,43401,43411,43491)
office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                     ,"99215")
IHD_icd_9_codes=c(410, 411, 412,413,414)
IHD_icd_10_codes=c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes=c(40,54,56,57,58,59,60,61,62,63,64,65,66,97,98,99)

primary_care_specialty_codes=c("01", "08", "11", "38")
surgery_specialty_codes=c("02","04","14","19","20","24","28","33","34","40","48","77","78","85","91")

#http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
#https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes=c("401","402","403","404","405")
hypertension_icd_10_codes=c("I10","I11","I12","I13","I15","I16")

#http://www.icd9data.com/2014/Volume1/460-519/490-496/default.htm
#https://www.icd10data.com/ICD10CM/Codes/J00-J99/J40-J47
copd_icd_9_codes=c("490","491","492","493","494","495","496")
copd_icd_10_codes=c("J40","J41","J42","J43","J44","J45","J47")

#http://www.icd9data.com/2015/Volume1/240-279/270-279/278/278.htm?__hstc=93424706.cdd51240e438a5219319ce13ccb23860.1648603374124.1648603374124.1648607295327.2&__hssc=93424706.9.1648607295327&__hsfp=908776442
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E65-E68/E66-
obesity_icd_9_codes=c("278")
obesity_icd_10_codes=c("E66")

#http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
#https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes=c("2962","2963")
depression_icd_10_codes=c("F32","F33")

#http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes=c("250")
diabetes_icd_10_codes=c("E08","E09","E10","E11","E13")

census_and_state_codes=read.csv("./results_mar/census_and_state_codes.csv")
census_and_state_codes[1:9,"state_code"] = paste("0",census_and_state_codes[1:9,"state_code"],sep="")

race_codes=data.frame(race_code = seq(0,6),
                      race = c("Unknown", "White", "Black", "Other", "Asian", "Hispanic", "North American Native")
                      )

sex_codes=data.frame(sex_code = seq(0,2),
                    sex = c("Unknown","Male","Female"))
                      
                      
                      

# Comparisons and analyses

## Comparisons between patients treated by integrated vs independent docotrs

I will compare the patients who were treated by the integrated vs non-integrated physicians, cardiologists and interventionists during the study period. Year 2013 did not divide cardiologists and inteventionists. So, We will not include this year in our comparisons of cardiologists and inteventionists.



## Reading the patient and physician integration results

In [146]:
yearly_calculations_stable_angina =
read_fst("results_may/yearly_calculations_stable_angina_with_integration.fst"
         ,as.data.table = T) 
yearly_calculations_unstable_angina =
read_fst("results_may/yearly_calculations_unstable_angina_with_integration.fst"
         ,as.data.table = T)
#physician_integration_stats = 
#read_fst("results_feb/physician_integration_stats.fst"
#        ,as.data.table = T)
#melted_physician_integration_stats=
#read_fst("results_feb/melted_physician_integration_stats.fst"
#        ,as.data.table = T)

In [147]:
tail(yearly_calculations_stable_angina,5)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

## Adding sex, race, and state names from codes

In [148]:
add_personal_details=function(data){
    
  require(tidyverse)
  require(dtplyr)
  require(lubridate)
  
  result=data %>%
  left_join(.,census_and_state_codes[,-1],by=c("state_code_at_diagnosis"="state_code"))%>%
  left_join(.,race_codes,by=c("race_code_at_diagnosis"="race_code"))%>%
  left_join(.,sex_codes,by=c("sex_code_at_diagnosis"="sex_code"))%>%
  mutate(
    age_group=case_when(
      age_at_diagnosis<75 & age_at_diagnosis>64 ~ "65-74",
      age_at_diagnosis>74 & age_at_diagnosis<85 ~ "75-84",
      age_at_diagnosis>84 ~ "85+"
    )
  )%>%
  as.data.table()
  
  return(result)
}

In [149]:
yearly_calculations_stable_angina=add_personal_details(yearly_calculations_stable_angina)
yearly_calculations_unstable_angina=add_personal_details(yearly_calculations_unstable_angina)

## Filtering the data for the IV model

In [150]:
data_for_modelling_filter=function(data){
  library(tidyverse)
  library(dtplyr)
  
  data%>%
  filter(state_code_at_diagnosis %!in% non_us_state_codes &
         age_at_diagnosis>=65&
         ((died_in_one_year_after_diagnosis==T & HMO_INDICATOR_sum>=floor(date_difference_diagnosis_death/30)-1) |
          (died_in_one_year_after_diagnosis==F & HMO_INDICATOR_sum==12))&
         ((died_in_one_year_after_diagnosis==T & ENTITLEMENT_BUY_IN_IND_sum>=floor(date_difference_diagnosis_death/30)-1) |
          (died_in_one_year_after_diagnosis==F & ENTITLEMENT_BUY_IN_IND_sum==12))&
         year_first_diagnosed!=2013&
         !is.na(most_common_cardiologist_PRF_PHYSN_NPI)
        )%>%
  as.data.table()
}



In [151]:
yearly_calculations_stable_angina=data_for_modelling_filter(yearly_calculations_stable_angina)
yearly_calculations_unstable_angina=data_for_modelling_filter(yearly_calculations_unstable_angina)

In [152]:
length(unique(yearly_calculations_stable_angina$DESY_SORT_KEY))

[1] 15309

In [153]:
length(unique(yearly_calculations_unstable_angina$DESY_SORT_KEY))

[1] 9683

In [154]:
yearly_calculations_stable_angina[is.na(angioplasty_with_cath),angioplasty_with_cath:=F]
yearly_calculations_unstable_angina[is.na(angioplasty_with_cath),angioplasty_with_cath:=F]
yearly_calculations_stable_angina[is.na(angioplasty_during_year_after_cath),angioplasty_during_year_after_cath:=F]
yearly_calculations_unstable_angina[is.na(angioplasty_during_year_after_cath),angioplasty_during_year_after_cath:=F]

In [155]:
colnames(yearly_calculations_stable_angina)

[1] "DESY_SORT_KEY"                                                                               "first_diagnosis"                                                                             "tot_allowed_carrier"                                                                        
  [4] "office_visit_count"                                                                          "office_visit_cost_carrier"                                                                   "cardiology_visit_count"                                                                     
  [7] "distinct_clinicians"                                                                         "distinct_cardiologists"                                                                      "distinct_primary_care_physicians"                                                           
 [10] "distinct_surgeons"                                                                           "distinct_other_specialties"                                                                  "catheterization_count"                                                                      
 [13] "catheterization_cost_carrier"                                                                "ecg_count"                                                                                   "ecg_cost_carrier"                                                                           
 [16] "cardiac_ct_count"                                                                            "cardiac_ct_cost_carrier"                                                                     "cardiac_mri_count"                                                                          
 [19] "cardiac_mri_cost_carrier"                                                                    "stress_test_count"                                                                           "stress_test_cost_carrier"                                                                   
 [22] "echocardiography_count"                                                                      "echocardiography_cost_carrier"                                                               "angioplasty_count"                                                                          
 [25] "angioplasty_cost_carrier"                                                                    "CABG_count"                                                                                  "CABG_cost_carrier"                                                                          
 [28] "stable_angina"                                                                               "unstable_angina"                                                                             "MI"                                                                                         
 [31] "cardiac_arrest"                                                                              "stroke"                                                                                      "hypertension"                                                                               
 [34] "copd"                                                                                        "obesity"                                                                                     "depression"                                                                                 
 [37] "diabetes"                                                                                    "icd_9_pure"                                                                                  "icd_10_pure"                                                                                
 [40] "catheterization_date"                                                                        "angioplasty_date"                                                                            "catheterization_doc_NPI"                                       

### Number of patients in each of stable and unstable angina groups after the filters

In [156]:
length(unique(yearly_calculations_stable_angina$DESY_SORT_KEY))
length(unique(yearly_calculations_unstable_angina$DESY_SORT_KEY))

[1] 15309

[1] 9683

In [157]:
sum(yearly_calculations_stable_angina[,most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS]<0.75,na.rm=T)
sum(yearly_calculations_unstable_angina[,most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS]<0.75,na.rm=T)

[1] 9679

[1] 6096

In [158]:
sum(yearly_calculations_stable_angina[,is.na(most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS)],na.rm=T)
sum(yearly_calculations_unstable_angina[,is.na(most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS)],na.rm=T)

[1] 4893

[1] 3108

In [159]:
sum(yearly_calculations_stable_angina[,most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS]>=0.75,na.rm=T)
sum(yearly_calculations_unstable_angina[,most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS]>=0.75,na.rm=T)

[1] 1985

[1] 1314

In [160]:
sum(yearly_calculations_stable_angina[,is.na(most_common_primary_care_physician_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS)],na.rm=T)
sum(yearly_calculations_unstable_angina[,is.na(most_common_primary_care_physician_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS)],na.rm=T)

[1] 5634

[1] 3522

## Naive models without IV

### Stable Angina

In [70]:
naive_model_stable_angina=glm(catheterization_count>0~
               (most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS<0.75):
               (most_common_primary_care_physician_avg_in_year_of_diagnosis_non_exclusive_HCPCS<0.75)+             
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_stable_angina,family="binomial"
              )



In [71]:
summary_naive_model_stable_angina=summ(naive_model_stable_angina)

In [72]:
data.frame(exp(summary_naive_model_stable_angina$coeftable))

,Est.,S.E.,z.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),10.6557041,1.331074,3.918523e+03,1.000000
sexMale,1.3217388,1.043057,7.477799e+02,1.000000
raceBlack,1.3989437,1.201541,6.224411e+00,1.069806
raceHispanic,1.2335501,1.281314,2.332046e+00,1.487559
raceNorth American Native,1.7011604,1.429776,4.419869e+00,1.147115
raceOther,1.4019438,1.259031,4.335218e+00,1.153082
raceUnknown,1.5355279,1.281781,5.627065e+00,1.087696
raceWhite,1.8334717,1.178810,3.984709e+01,1.000229
age_at_diagnosis,0.9545719,1.003045,2.288645e-07,1.000000


### Unstable Angina

In [73]:
naive_model_unstable_angina=lm(catheterization_count>0~
               (most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>0.5)+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_unstable_angina
              )

In [74]:
summary_naive_model_unstable_angina=summ(naive_model_unstable_angina)

In [75]:
data.frame(summary_naive_model_unstable_angina$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.335258544,0.0746908009,17.87714858,4.563972e-70
most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS > 0.5TRUE,0.025026422,0.0131437965,1.90404821,5.694112e-02
sexMale,0.041387787,0.0112272016,3.68638499,2.290249e-04
raceBlack,-0.004800195,0.0479503455,-0.10010762,9.202615e-01
raceHispanic,-0.006295510,0.0631144104,-0.09974758,9.205473e-01
raceNorth American Native,-0.041495383,0.0991325992,-0.41858464,6.755314e-01
raceOther,0.016210292,0.0652312619,0.24850496,8.037504e-01
raceUnknown,-0.038557847,0.0655575630,-0.58815253,5.564472e-01
raceWhite,0.095383554,0.0432661595,2.20457639,2.751328e-02


## The IV model

### Stable Angina

#### Stage 1

In [161]:
yearly_calculations_stable_angina[,most_common_primary_care_physician_2013_integrated_in_year_of_diagnosis
                                  :=(most_common_primary_care_physician_2013_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.75)
                                 ]

In [163]:
yearly_calculations_stable_angina$race=as.factor(yearly_calculations_stable_angina$race)
yearly_calculations_stable_angina$census_region=as.factor(yearly_calculations_stable_angina$census_region)

In [164]:
#relevel the data for the model so the right level is used within the model as a reference
yearly_calculations_stable_angina$race=relevel(yearly_calculations_stable_angina$race,ref="White")
yearly_calculations_stable_angina$census_region=relevel(yearly_calculations_stable_angina$census_region,ref="Northeast")

In [165]:
iv_model_stable_angina_stage_1=lm((most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.75)~
               most_common_primary_care_physician_2013_integrated_in_year_of_diagnosis+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75,]
              )

In [166]:
summary_iv_model_stable_angina_stage_1=summ(iv_model_stable_angina_stage_1,robust = "HC3")

In [203]:
summary_iv_model_stable_angina_stage_1

MODEL INFO:
Observations: 6830 (2849 missing obs. deleted)
Dependent Variable: (most_common_cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS >= 0.75)
Type: OLS linear regression 

MODEL FIT:
F(18,6811) = 11.91, p = 0.00
R² = 0.03
Adj. R² = 0.03 

Standard errors: Robust, type = HC3
----------------------------------------------------------------------------------------------------------------
                                                                                     Est.   S.E.   t val.      p
--------------------------------------------------------------------------------- ------- ------ -------- ------
(Intercept)                                                                          0.12   0.04     2.91   0.00
most_common_primary_care_physician_2013_integrated_in_year_of_diagnosisTRUE          0.23   0.03     7.08   0.00
sexMale                                                                              0.01   0.01     1.52   0.13
raceAsian                     

In [167]:
data.frame(summary_iv_model_stable_angina_stage_1$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.215585e-01,0.0417891536,2.90885400,3.639293e-03
most_common_primary_care_physician_2013_integrated_in_year_of_diagnosisTRUE,2.347111e-01,0.0331631345,7.07746906,1.614870e-12
sexMale,1.097497e-02,0.0072317651,1.51760568,1.291602e-01
raceAsian,-2.260541e-02,0.0198188510,-1.14060126,2.540760e-01
raceBlack,-4.318736e-03,0.0141302006,-0.30563870,7.598891e-01
raceHispanic,-3.680797e-02,0.0217360980,-1.69340300,9.042451e-02
raceNorth American Native,1.751342e-01,0.1027241304,1.70489820,8.825902e-02
raceOther,-1.628337e-03,0.0259780867,-0.06268117,9.500222e-01
raceUnknown,4.194452e-02,0.0531004186,0.78990940,4.296082e-01


#### Predictions

In [168]:
predictions_iv_model_stable_angina_stage_1=predict(iv_model_stable_angina_stage_1,
        newdata = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75,
                                                    .(most_common_primary_care_physician_2013_integrated_in_year_of_diagnosis,
                                                    sex,
                                                    race,
                                                    age_at_diagnosis,
                                                    census_region,
                                                    hypertension,
                                                    copd,
                                                    obesity,
                                                    depression,
                                                    diabetes,
                                                    wscore)])

In [169]:
head(predictions_iv_model_stable_angina_stage_1)

1          2          3          4          5          6 
0.07044459 0.10839922 0.09674915 0.09497972 0.07275991         NA

In [170]:
yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75,predictions_iv:=predictions_iv_model_stable_angina_stage_1]



#### Stage 2

##### Catheterization

In [198]:
iv_model_stable_angina_stage_2_catheterization=lm(catheterization_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore+
                    MI,
                    data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [199]:
summary_iv_model_stable_angina_stage_2_catheterization=summ(iv_model_stable_angina_stage_2_catheterization,robust = "HC3")

In [202]:
summary_iv_model_stable_angina_stage_2_catheterization

MODEL INFO:
Observations: 6833 (2846 missing obs. deleted)
Dependent Variable: catheterization_count > 0
Type: OLS linear regression 

MODEL FIT:
F(19,6813) = 34.61, p = 0.00
R² = 0.09
Adj. R² = 0.09 

Standard errors: Robust, type = HC3
--------------------------------------------------------------
                                   Est.   S.E.   t val.      p
------------------------------- ------- ------ -------- ------
(Intercept)                        1.24   0.07    19.02   0.00
predictions_iv                     0.34   0.15     2.30   0.02
sexMale                            0.04   0.01     3.90   0.00
raceAsian                         -0.09   0.03    -2.77   0.01
raceBlack                         -0.07   0.02    -3.07   0.00
raceHispanic                      -0.02   0.05    -0.52   0.60
raceNorth American Native          0.02   0.12     0.14   0.89
raceOther                         -0.08   0.04    -1.82   0.07
raceUnknown                       -0.06   0.07    -0.87   0.38
age_at

In [200]:
data.frame(summary_iv_model_stable_angina_stage_2_catheterization$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.240575418,0.0652384120,19.0160272,1.328330e-78
predictions_iv,0.336468792,0.1465396826,2.2960934,2.170076e-02
sexMale,0.044883471,0.0115116948,3.8989455,9.754034e-05
raceAsian,-0.092487380,0.0333362291,-2.7743804,5.545761e-03
raceBlack,-0.070914232,0.0231196538,-3.0672705,2.168708e-03
raceHispanic,-0.024137159,0.0463212938,-0.5210813,6.023271e-01
raceNorth American Native,0.016384466,0.1152423970,0.1421739,8.869468e-01
raceOther,-0.075112802,0.0412900571,-1.8191499,6.893244e-02
raceUnknown,-0.061319060,0.0705294098,-0.8694112,3.846529e-01


##### Echo

In [174]:
iv_model_stable_angina_stage_2_echocardiography=lm(echocardiography_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [175]:
summary_iv_model_stable_angina_stage_2_echocardiography=summ(iv_model_stable_angina_stage_2_echocardiography,robust = "HC3")

In [176]:
data.frame(summary_iv_model_stable_angina_stage_2_echocardiography$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.415274459,0.0707768785,5.86737460,4.635114e-09
predictions_iv,-0.074332555,0.1504751377,-0.49398563,6.213323e-01
sexMale,-0.005569178,0.0119887472,-0.46453377,6.422802e-01
raceAsian,0.106915725,0.0398196124,2.68500164,7.270504e-03
raceBlack,0.069949840,0.0237910357,2.94017631,3.291280e-03
raceHispanic,0.003423934,0.0483369633,0.07083469,9.435314e-01
raceNorth American Native,0.130114887,0.1045839894,1.24411860,2.134987e-01
raceOther,0.087287945,0.0449642804,1.94127302,5.226635e-02
raceUnknown,0.084136316,0.0704329373,1.19455924,2.323009e-01


##### Stress test

In [177]:
iv_model_stable_angina_stage_2_stress_test=lm(stress_test_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [178]:
summary_iv_model_stable_angina_stage_2_stress_test=summ(iv_model_stable_angina_stage_2_stress_test,robust = "HC3")

In [179]:
data.frame(summary_iv_model_stable_angina_stage_2_stress_test$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),6.599780e-01,0.0655541458,10.06767773,1.123234e-23
predictions_iv,6.084844e-03,0.1402205301,0.04339481,9.653881e-01
sexMale,8.910383e-03,0.0114539613,0.77793021,4.366372e-01
raceAsian,-3.377053e-02,0.0401315773,-0.84149510,4.001002e-01
raceBlack,5.313951e-04,0.0236288297,0.02248927,9.820583e-01
raceHispanic,-4.840251e-03,0.0485199817,-0.09975789,9.205395e-01
raceNorth American Native,4.219389e-02,0.1185838532,0.35581477,7.219903e-01
raceOther,4.417078e-02,0.0462153616,0.95575972,3.392275e-01
raceUnknown,3.080935e-02,0.0736751879,0.41817805,6.758301e-01


##### CABG

In [180]:
iv_model_stable_angina_stage_2_CABG=lm(CABG_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [181]:
summary_iv_model_stable_angina_stage_2_CABG=summ(iv_model_stable_angina_stage_2_CABG,robust = "HC3")

In [182]:
data.frame(summary_iv_model_stable_angina_stage_2_CABG$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.205459374,0.0275948755,7.4455627,1.084230e-13
predictions_iv,0.034176875,0.0685727057,0.4984035,6.182158e-01
sexMale,0.041804059,0.0051459621,8.1236625,5.326932e-16
raceAsian,-0.029066146,0.0089259095,-3.2563792,1.133948e-03
raceBlack,-0.030551411,0.0076354509,-4.0012582,6.367409e-05
raceHispanic,-0.018996869,0.0161387324,-1.1770980,2.391975e-01
raceNorth American Native,0.054487766,0.0708240555,0.7693398,4.417183e-01
raceOther,-0.011449205,0.0177803047,-0.6439263,5.196449e-01
raceUnknown,-0.046888654,0.0235312834,-1.9926093,4.634407e-02


##### Angioplasty

In [195]:
iv_model_stable_angina_stage_2_angioplasty=lm(angioplasty_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore+
                    MI,
                    data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [196]:
summary_iv_model_stable_angina_stage_2_angioplasty=summ(iv_model_stable_angina_stage_2_angioplasty,robust = "HC3")

In [201]:
summary_iv_model_stable_angina_stage_2_angioplasty

MODEL INFO:
Observations: 6833 (2846 missing obs. deleted)
Dependent Variable: angioplasty_count > 0
Type: OLS linear regression 

MODEL FIT:
F(19,6813) = 27.80, p = 0.00
R² = 0.07
Adj. R² = 0.07 

Standard errors: Robust, type = HC3
--------------------------------------------------------------
                                   Est.   S.E.   t val.      p
------------------------------- ------- ------ -------- ------
(Intercept)                        0.34   0.05     7.02   0.00
predictions_iv                     0.21   0.12     1.76   0.08
sexMale                            0.05   0.01     6.05   0.00
raceAsian                         -0.02   0.02    -0.87   0.38
raceBlack                         -0.05   0.02    -3.04   0.00
raceHispanic                       0.02   0.04     0.59   0.56
raceNorth American Native          0.06   0.10     0.59   0.56
raceOther                          0.01   0.03     0.25   0.81
raceUnknown                        0.02   0.06     0.35   0.72
age_at_dia

In [197]:
data.frame(summary_iv_model_stable_angina_stage_2_angioplasty$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.335852240,0.0478749136,7.0152030,2.517454e-12
predictions_iv,0.206450643,0.1172280501,1.7611028,7.826586e-02
sexMale,0.050855173,0.0084083660,6.0481636,1.542603e-09
raceAsian,-0.020728517,0.0237575662,-0.8725017,3.829655e-01
raceBlack,-0.047106107,0.0155104202,-3.0370620,2.398003e-03
raceHispanic,0.020786212,0.0353979665,0.5872149,5.570789e-01
raceNorth American Native,0.058945145,0.1003223463,0.5875575,5.568488e-01
raceOther,0.007685169,0.0312063370,0.2462695,8.054810e-01
raceUnknown,0.020148774,0.0570238621,0.3533393,7.238450e-01


##### Angioplasty after cath

In [186]:
iv_model_stable_angina_stage_2_angio_with_cath=lm(angioplasty_during_year_after_cath==T~
                                                  predictions_iv+
                                                  sex+
                                                  race+
                                                  age_at_diagnosis+
                                                  census_region+
                                                  hypertension+
                                                  copd+
                                                  obesity+
                                                  depression+
                                                  diabetes+
                                                  wscore,
                                                  data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                                                 )

In [187]:
summary_iv_model_stable_angina_stage_2_angio_with_cath=summ(iv_model_stable_angina_stage_2_angio_with_cath,robust = "HC3")

In [188]:
data.frame(summary_iv_model_stable_angina_stage_2_angio_with_cath$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.306852774,0.0478414675,6.4139499,1.512930e-10
predictions_iv,0.258139121,0.1214224535,2.1259587,3.354255e-02
sexMale,0.048946683,0.0083918965,5.8326128,5.706514e-09
raceAsian,-0.021256978,0.0242162274,-0.8777989,3.800838e-01
raceBlack,-0.039867411,0.0154284554,-2.5840183,9.786279e-03
raceHispanic,0.022018366,0.0362421025,0.6075355,5.435158e-01
raceNorth American Native,0.032313582,0.0995749658,0.3245151,7.455580e-01
raceOther,0.006879010,0.0326401735,0.2107529,8.330864e-01
raceUnknown,0.024871799,0.0596397955,0.4170336,6.766670e-01


##### Death after angioplasty

In [189]:
iv_model_stable_angina_stage_2_angio_with_cath=lm(MI==T~
                                                  predictions_iv+
                                                  sex+
                                                  race+
                                                  age_at_diagnosis+
                                                  census_region+
                                                  hypertension+
                                                  copd+
                                                  obesity+
                                                  depression+
                                                  diabetes+
                                                  wscore,
                                                  data = yearly_calculations_stable_angina[most_common_primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75&
                                                                                          angioplasty_with_cath==T]
                                                 )

In [190]:
summary_iv_model_stable_angina_stage_2_angio_with_cath=summ(iv_model_stable_angina_stage_2_angio_with_cath,robust = "HC3")

In [191]:
data.frame(summary_iv_model_stable_angina_stage_2_angio_with_cath$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.093721835,0.197333404,-0.47494156,6.349669e-01
predictions_iv,0.393812628,0.316880344,1.24278023,2.143372e-01
sexMale,-0.003779600,0.032888518,-0.11492157,9.085380e-01
raceAsian,0.117838990,0.182310919,0.64636277,5.182422e-01
raceBlack,0.042072946,0.079658145,0.52816878,5.975384e-01
raceHispanic,0.109961442,0.151154810,0.72747564,4.671617e-01
raceNorth American Native,-0.358527190,0.203263105,-1.76385769,7.816303e-02
raceOther,-0.103872661,0.104737941,-0.99173862,3.216449e-01
raceUnknown,0.036473613,0.140816897,0.25901446,7.956952e-01


### Unstable Angina

#### Stage 1

In [150]:
yearly_calculations_unstable_angina[,primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half
                                  :=(primary_care_physician_2013_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.6)
                                 ]

In [151]:
iv_model_unstable_angina_stage_1=lm((cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.6)~
               primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2,]
              )

In [152]:
summary_iv_model_unstable_angina_stage_1=summ(iv_model_unstable_angina_stage_1,robust = "HC3")

In [153]:
data.frame(summary_iv_model_unstable_angina_stage_1$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.170199142,0.0731688368,2.32611518,2.006093e-02
primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_halfTRUE,0.228572083,0.0467151455,4.89289031,1.031820e-06
sexMale,0.026997849,0.0112135256,2.40761468,1.610103e-02
raceBlack,0.020060818,0.0381112343,0.52637544,5.986559e-01
raceHispanic,-0.030708970,0.0443526348,-0.69238209,4.887366e-01
raceNorth American Native,0.144051039,0.1352543226,1.06503834,2.869213e-01
raceOther,0.002866352,0.0490503462,0.05843693,9.534034e-01
raceUnknown,-0.007221504,0.0724412873,-0.09968768,9.205972e-01
raceWhite,0.046220281,0.0326522949,1.41552933,1.569893e-01


#### Predictions

In [154]:
predictions_iv_model_unstable_angina_stage_1=predict(iv_model_unstable_angina_stage_1,
        newdata = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2,
                                                    .(primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half,
                                                    sex,
                                                    race,
                                                    age_at_diagnosis,
                                                    census_region,
                                                    hypertension,
                                                    copd,
                                                    obesity,
                                                    depression,
                                                    diabetes,
                                                    wscore)])

In [155]:
head(predictions_iv_model_unstable_angina_stage_1)

1          2          3          4          5          6 
0.13858338 0.13716212 0.08239007 0.13973876         NA 0.05930227

In [156]:
yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2,predictions_iv:=predictions_iv_model_unstable_angina_stage_1]

#### Stage 2

##### Catheterization

In [157]:
iv_model_unstable_angina_stage_2_catheterization=lm(catheterization_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2]
                   )

In [158]:
summary_iv_model_unstable_angina_stage_2_catheterization=summ(iv_model_unstable_angina_stage_2_catheterization,robust = "HC3")

In [159]:
data.frame(summary_iv_model_unstable_angina_stage_2_catheterization$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.591985057,0.114545796,13.8982408,6.028080e-43
predictions_iv,0.228767518,0.217634004,1.0511571,2.932484e-01
sexMale,0.025352219,0.016344700,1.5510972,1.209556e-01
raceBlack,-0.147255646,0.069193022,-2.1281864,3.338110e-02
raceHispanic,-0.249757467,0.091098833,-2.7416100,6.140406e-03
raceNorth American Native,-0.240245343,0.154674262,-1.5532341,1.204444e-01
raceOther,-0.126814353,0.091728502,-1.3824967,1.668945e-01
raceUnknown,-0.225130669,0.115669674,-1.9463241,5.168403e-02
raceWhite,-0.056089367,0.063290258,-0.8862243,3.755486e-01


##### Echo

In [100]:
iv_model_unstable_angina_stage_2_echocardiography=lm(echocardiography_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [101]:
summary_iv_model_unstable_angina_stage_2_echocardiography=summ(iv_model_unstable_angina_stage_2_echocardiography,robust = "HC3")

In [102]:
data.frame(summary_iv_model_unstable_angina_stage_2_echocardiography$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.2535125379,0.116332316,2.17920992,2.937145e-02
predictions_iv,-0.1917779739,0.196810555,-0.97442931,3.298995e-01
sexMale,0.0186825457,0.014854038,1.25774189,2.085550e-01
raceBlack,-0.0261798241,0.065244102,-0.40125963,6.882494e-01
raceHispanic,0.0348960360,0.084801138,0.41150434,6.807238e-01
raceNorth American Native,0.0239281752,0.162055240,0.14765444,8.826226e-01
raceOther,0.0242055893,0.081735011,0.29614713,7.671324e-01
raceUnknown,0.0640591655,0.108728559,0.58916596,5.557816e-01
raceWhite,-0.0008222624,0.059552134,-0.01380744,9.889843e-01


##### CABG

In [103]:
iv_model_unstable_angina_stage_2_CABG=lm(CABG_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [104]:
summary_iv_model_unstable_angina_stage_2_CABG=summ(iv_model_unstable_angina_stage_2_CABG,robust = "HC3")

In [105]:
data.frame(summary_iv_model_unstable_angina_stage_2_CABG$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.375835931,0.0537227543,6.9958426,3.054795e-12
predictions_iv,-0.180735917,0.0770558261,-2.3455192,1.904687e-02
sexMale,0.049835240,0.0080415745,6.1971993,6.299932e-10
raceBlack,0.004795382,0.0285594967,0.1679085,8.666634e-01
raceHispanic,-0.028427136,0.0357253268,-0.7957138,4.262433e-01
raceNorth American Native,0.138752358,0.1161342677,1.1947581,2.322490e-01
raceOther,0.020060964,0.0415788292,0.4824803,6.294900e-01
raceUnknown,-0.065034452,0.0479793323,-1.3554681,1.753413e-01
raceWhite,0.031565942,0.0253297272,1.2462014,2.127599e-01


##### Angioplasty

In [109]:
iv_model_unstable_angina_stage_2_angioplasty=lm(angioplasty_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [110]:
summary_iv_model_unstable_angina_stage_2_angioplasty=summ(iv_model_unstable_angina_stage_2_angioplasty,robust = "HC3")

In [111]:
data.frame(summary_iv_model_unstable_angina_stage_2_angioplasty$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.3694929284,0.1031784351,3.58110615,3.460153e-04
predictions_iv,0.1118077616,0.1756437978,0.63655969,5.244464e-01
sexMale,0.0760251704,0.0130761593,5.81402906,6.551324e-09
raceBlack,-0.1727680646,0.0607122867,-2.84568534,4.453061e-03
raceHispanic,-0.1316956000,0.0758641589,-1.73593963,8.264783e-02
raceNorth American Native,-0.1254717592,0.1377896464,-0.91060368,3.625565e-01
raceOther,0.0050744317,0.0834263852,0.06082526,9.515013e-01
raceUnknown,-0.1275061379,0.0982838616,-1.29732528,1.945905e-01
raceWhite,-0.0614355609,0.0582006503,-1.05558203,2.912199e-01
